inladen van alle foto id's in een dataframe met bijbehorende label uit map naam <br>
predictions van de ensemble methode op val_gen<br>
deze predicties als library in een dataframe toevoegen <br>
Dan for loop checken welke nummers niet kloppen

In [29]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_curve, auc

map0 = r"C:\Users\20192735\Documents\8P361_images\train+val\valid\0"
map1 = r"C:\Users\20192735\Documents\8P361_images\train+val\valid\1"
filenames0 = os.listdir(map0)
filenames1 = os.listdir(map1)
zeros = [0]*len(filenames0)
ones = [1]*len(filenames1)

df_true_0 = pd.DataFrame(filenames0, columns=["filename"])
df_true_0["true_label"]=zeros
df_true_1 = pd.DataFrame(filenames1, columns=["filename"])
df_true_1["true_label"]=ones

df_true = pd.concat([df_true_0,df_true_1])
df_true= df_true.set_index("filename")

df_true.head()


,true_label
filename,
0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c.jpg,0
0007da302a322ef9444dc55227536cbb61e18781.jpg,0
0033eb6f1ae535166311e5837b006cba336e9200.jpg,0
003ff6ef7139d48052a5f60adb08304325765a9f.jpg,0
0043f20e46038846be8aea6cbacf75f7f17716ee.jpg,0


In [41]:
df_inception_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\Inception_submission.csv')
df_efficient_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\EfficientNet_submission.csv')
df_resnet_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\ResNet_submission.csv')
df_mobilenet_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\MobileNetV2_submission.csv')
df_xception_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\Xception_submission.csv')

df_inception = df_inception_old.rename(columns={'label':'label_inception'}, errors='raise')
df_efficient = df_efficient_old.rename(columns={'label':'label_efficient'}, errors='raise')
df_resnet = df_resnet_old.rename(columns={'label':'label_resnet'}, errors='raise')
df_mobilenet = df_mobilenet_old.rename(columns={'label':'label_mobilenet'}, errors='raise')
df_xception = df_xception_old.rename(columns={'label':'label_xception'}, errors='raise')

all_probs = pd.DataFrame(data=[df_inception['id'], df_inception['label_inception'], df_xception['label_xception'], df_efficient['label_efficient'], df_resnet['label_resnet'], df_mobilenet['label_mobilenet']])
all_probs = all_probs.transpose()
all_probs["filename"]= all_probs["id"]+".jpg"
all_probs= all_probs.set_index("filename")
all_probs.head()


,id,label_inception,label_xception,label_efficient,label_resnet,label_mobilenet
filename,,,,,,
00006537328c33e284c973d7b39d340809f7271b.jpg,00006537328c33e284c973d7b39d340809f7271b,0.979511,0.992646,0.979737,0.994229,0.996166
0000ec92553fda4ce39889f9226ace43cae3364e.jpg,0000ec92553fda4ce39889f9226ace43cae3364e,0.966265,0.933859,0.955977,0.99346,0.66574
00024a6dee61f12f7856b0fc6be20bc7a48ba3d2.jpg,00024a6dee61f12f7856b0fc6be20bc7a48ba3d2,0.961354,0.901285,0.830304,0.946857,0.998542
000253dfaa0be9d0d100283b22284ab2f6b643f6.jpg,000253dfaa0be9d0d100283b22284ab2f6b643f6,0.516929,0.869525,0.900826,0.167476,0.995677
000270442cc15af719583a8172c87cd2bd9c7746.jpg,000270442cc15af719583a8172c87cd2bd9c7746,0.001821,0.002915,0.006509,0.024789,0.000119


In [44]:
true_and_pred = df_true.join(all_probs)
true_and_pred.head()

,true_label,id,label_inception,label_xception,label_efficient,label_resnet,label_mobilenet
filename,,,,,,,
0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN
0007da302a322ef9444dc55227536cbb61e18781.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN
0033eb6f1ae535166311e5837b006cba336e9200.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN
003ff6ef7139d48052a5f60adb08304325765a9f.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN
0043f20e46038846be8aea6cbacf75f7f17716ee.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
test_df = pd.DataFrame({'filename': ["a","b","c","d","e","f"],
                   'true_label': [0,0,0,1,1,1], "ensemble_A" : [1,1,1,1,1,1]})
test_df=test_df.set_index("filename")
test_df.head()


y_true = test_df["true_label"]
y_pred = test_df["ensemble_A"]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
# TP , FP
# FN , TN
FP_A = cm[0][1]
TN_A = cm[1][1]
FPR_A = FP_A/(FP_A+ TN_A)
print(FPR_A)

0.5
